In [2]:
import os
import gym                                                      # Build envs and load pre-existing envs        
from stable_baselines3 import PPO                               # RL Algorithm
from stable_baselines3.common.vec_env import DummyVecEnv        # Vectorize environments
from stable_baselines3.common.evaluation import evaluate_policy

import warnings
def warn(*args, **kwargs):
    pass
warnings.warn = warn

## Loading an environment

In [3]:
environment_name = 'CartPole-v1'
env = gym.make(environment_name, render_mode="human")

In [3]:
episodes = 5

for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0

    while not done:
        env.render()
        action = env.action_space.sample()
        next_obs_state, reward, done, trunc, info = env.step(action)
        score += reward

    print(f"Episode: {episode} | Score: {score}")
env.close()

Episode: 1 | Score: 15.0
Episode: 2 | Score: 29.0
Episode: 3 | Score: 25.0
Episode: 4 | Score: 12.0
Episode: 5 | Score: 25.0


## Understanding the Environment

In [4]:
env.action_space

Discrete(2)

In [5]:
env.observation_space

Box([-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38], [4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38], (4,), float32)

## Train an RL Model

In [5]:
log_path = os.path.join('training', 'logs')

In [7]:
env = gym.make(environment_name, render_mode='human')
env = DummyVecEnv([lambda: env])
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path)

Using cpu device


In [8]:
model.learn(total_timesteps=20000)

Logging to training\logs\PPO_3
-----------------------------
| time/              |      |
|    fps             | 46   |
|    iterations      | 1    |
|    time_elapsed    | 43   |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 44          |
|    iterations           | 2           |
|    time_elapsed         | 92          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.008523986 |
|    clip_fraction        | 0.102       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.686      |
|    explained_variance   | -0.00746    |
|    learning_rate        | 0.0003      |
|    loss                 | 6.95        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0157     |
|    value_loss           | 49.8        |
-----------------------------------------
---

## Save and load the model

In [9]:
save_path = os.path.join('training', 'saved_models', 'PPO_model_cartpole')

In [10]:
model.save(save_path)

In [11]:
loaded_model = PPO.load(save_path, env=env)
loaded_model

## Evaluation

In [12]:
print(evaluate_policy(model, env, n_eval_episodes=10, render=True))     # Mean / Std (time upright)

(500.0, 0.0)


## Testing

In [13]:
episodes = 5

for episode in range(1, episodes+1):
    obs = env.reset()
    done = False
    score = 0

    while not done:
        env.render()
        action, _ = model.predict(obs)
        obs, reward, done, info = env.step(action)
        score += reward

    print(f"Episode: {episode} | Score: {score}")
env.close()

Episode: 1 | Score: [38.]
Episode: 2 | Score: [500.]
Episode: 3 | Score: [107.]
Episode: 4 | Score: [500.]
Episode: 5 | Score: [500.]


## Viewing Logs in Tensorboard

Run the following command in the terminal and open `localhost:6006`<br>
```
tensorboard --logdir '<log_path>'
```

## Adding a Callback to the training stage

In [11]:
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold

In [12]:
saved_models_path = os.path.join('training', 'saved_models')

stop_callback = StopTrainingOnRewardThreshold(reward_threshold=300, verbose=1)
eval_callback = EvalCallback(env,
                             callback_on_new_best=stop_callback,
                             eval_freq=10000,
                             best_model_save_path=saved_models_path,
                             verbose=1)

In [13]:
env = gym.make(environment_name, render_mode='human')
new_model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path)

env.reset()
env.render()
new_model.learn(total_timesteps=20000, callback=eval_callback)
env.close()

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Logging to training\logs\PPO_4
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 20       |
|    ep_rew_mean     | 20       |
| time/              |          |
|    fps             | 46       |
|    iterations      | 1        |
|    time_elapsed    | 43       |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 25.2        |
|    ep_rew_mean          | 25.2        |
| time/                   |             |
|    fps                  | 46          |
|    iterations           | 2           |
|    time_elapsed         | 88          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.009053401 |
|    clip_fraction        | 0.0989      |
|    clip_range           | 0.2

## Using a different algorithm

In [24]:
from stable_baselines3 import DQN

env = gym.make(environment_name, render_mode='human')
dqn_model = DQN('MlpPolicy', env, verbose=1, tensorboard_log=log_path)

env.reset()
env.render()
dqn_model.learn(total_timesteps=20000)
env.close()

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Logging to training\logs\DQN_2
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 15.8     |
|    ep_rew_mean      | 15.8     |
|    exploration_rate | 0.97     |
| time/               |          |
|    episodes         | 4        |
|    fps              | 45       |
|    time_elapsed     | 1        |
|    total_timesteps  | 63       |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 19.6     |
|    ep_rew_mean      | 19.6     |
|    exploration_rate | 0.925    |
| time/               |          |
|    episodes         | 8        |
|    fps              | 46       |
|    time_elapsed     | 3        |
|    total_timesteps  | 157      |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.601    |
|    n_updates        | 14       |
